In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import os
from prettytable import PrettyTable
from pylab import *
from scipy.stats import wasserstein_distance
import random
import csv
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
# MaryClare's
#os.chdir('/Users/maryclaremartin/Documents/jup/ExtraSensory')

# Josh's
#os.chdir("D:/Research_Projects/REU2021-human-context-recognition/UCI_data")

# Harrison's
os.chdir("/Users/hkimr/Desktop/WPI Github/REU2021-human-context-recognition/UCI_data")

#os.chdir('../')
softmax = nn.Softmax(dim = 1)

In [4]:
##load and scale data
#returns scaled data (X) and labels (Y)
#label: array, list of activities to use (represented by integers)
#users: array, list of users whose data is to be used (represented by integers)

def start_data(label, users, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names

    #X_train = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z'] #Selecting only acceleration columns
    
    #X_train_1 = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z']
    #X_train_2 = data.loc[:,'81 tBodyAccJerk-mean()-X':'160 tBodyGyro-correlation()-Y,Z']
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']

    y_train_subject = pd.read_csv('data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X = GAN_data.iloc[:, :-2].values
    #X = GAN_data.loc[:,'1 tBodyAcc-mean()-X':'160 tBodyGyro-correlation()-Y,Z'].values
    y = GAN_data[['Activity']].values
    
    return X, y

In [5]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim).to(device)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

##calculates generator loss
#gen: generator
#disc: discriminator
#criterion1: loss function1
#criterion2: loss function2
#batch_size: batch size
#z_dim: number of dimensions in the latent space
def get_gen_loss(gen, disc, criterion1, batch_size, z_dim):
    latent_vectors = get_noise(batch_size, z_dim)
    fake_features = gen(latent_vectors)
    
    disc.eval()
    pred_disc = disc(fake_features)
    disc.train()
    
    d_loss = criterion1(pred_disc, torch.ones_like(pred_disc))
        
    return d_loss

In [6]:
def discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

#defines discriminator class
class Discriminator(nn.Module):
    def __init__(self, feature_dim = input_shape, hidden_dim = 16):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            discriminator_block(feature_dim, 25),
            discriminator_block(25, 10),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )
    def forward(self, feature_vector):
        return self.disc(feature_vector)
    
def get_disc_loss(gen, disc, criterion, real_features, batch_size, z_dim):
    latent_vectors = get_noise(batch_size, z_dim)
    
    gen.eval()
    fake_features = gen(latent_vectors)
    gen.train()
    
    pred_fake = disc(fake_features.detach())
    
    ground_truth = torch.zeros_like(pred_fake)
    loss_fake = criterion(pred_fake, ground_truth)
    
    pred_real = disc(real_features)
    ground_truth = torch.ones_like(pred_real)
    loss_real = criterion(pred_real, ground_truth)
    
    disc_loss = (loss_fake + loss_real) / 2
    return disc_loss

In [7]:
##calculates performance statistics for each epoch of training
#gen: generator
#disc: discriminator
#b_size: batch size
#z_dim: number of dimensions of the latent space
##returns accuracy, precision, recall, fpR, and f1 score
def performance_stats(gen, disc, b_size, z_dim, batch = None):
    tp = 0 #true positive
    fp = 0 #false positive
    tn = 0 #true negative
    fn = 0 #false negative

    with torch.no_grad():
        y_hat_fake = torch.Tensor([0])
        y_hat_real = torch.Tensor([0])
        
        if batch is None:
            print('Error')
        else:
            latent_vectors = get_noise(b_size, z_dim)
            gen.eval()
            fake_features = gen(latent_vectors)
            gen.train()
            disc.eval()
            y_hat_fake = torch.round(disc(fake_features))
            y_hat_real = torch.round(disc(batch))
            disc.train()
         
        fpR = torch.mean(y_hat_fake)
        recall = torch.mean(y_hat_real)
        class_acc = ((len(y_hat_fake) - torch.sum(y_hat_fake)) + torch.sum(y_hat_real)) / (2*b_size)
        #print(f'Classification Accuracy: {class_acc:.2f}'
        #print(f'Recall: {recall:.2f}') #What percent of the true positives were identified
        return class_acc, recall, fpR
    
def save_model(gen, disc, model_name):
    torch.save(gen.state_dict(), f"saved_models/{model_name}_gen.param")
    torch.save(disc.state_dict(), f"saved_models/{model_name}_disc.param")

In [8]:
###initalize parameters that depend on training loop parameters
#X: acceleration data
#y: labels associated with X data (fake or real)
#z_dim: number of dimensions to the latent space
#disc_lr: discriminator learning rate
#gen_lr: generator learning rate
#DISCRIMINATOR: 1 to indicate if discriminator is training
#batch_size: batch size
#disc: initialized discrimiantor

def initialize_params(X, y, z_dim, disc_lr, gen_lr, DISCRIMINATOR, batch_size, disc):
    #initialize generator
    gen = Generator(z_dim).to(device)
    #indicate that discriminator is training
    to_train = DISCRIMINATOR
    #create training features
    train_features = torch.tensor(X)
    #create training labels
    train_labels = torch.tensor(y)
    #concatenate to create training data
    train_data = torch.utils.data.TensorDataset(train_features, train_labels)
    #create data loader for training data
    train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
    #initialize generator and discriminator optimizers
    opt_disc = optim.Adam(disc.parameters(), lr = disc_lr)
    opt_gen = optim.Adam(gen.parameters(), lr = gen_lr)
    
    return gen, to_train, train_features, train_labels, train_data, train_loader, opt_disc, opt_gen   

In [9]:
######Training loop to train GAN

#Parameters to specifiy: 
    #X: starting accelerometer data
    #y: starting labels for X data (fake or real)
    
#Set parameters (do not change)
    #criterion: loss function (BCE)
    #dig: number of significant digits for printing (5)
    #feature_dim: Number of dimensions of output from generator (26)
    #GENERATOR: set generator to zero for training
    #DISCRIMINATOR: set discriminator to one for training
    #train_string: starting machine to train (DISC)
    #disc: initalize discriminator
    #rel_epochs: Epochs passed since last switch (constant training) (0)
    #rows: initialization of array to save data of each epoch to CSV file ([])
    #heading: array of column headings for table (["Epoch", "Machine Training", "Discriminator Loss", 
                    #"Generator Loss", "FPR", "Recall", "Median Wasserstein", "Mean Wasserstein"])
    #table: intialize a table as a pretty table to save epoch data
    #switch_count: number of switches in dynamic training (0)
    
#Set parameters (can change):
    #z_dim: number of dimensions of latent vector (100)
    #gen_lr: generator learning rate (.001)
    #disc_lr: discriminator learning rate (.001) (shoud be equal to gen_lr)
    #batch_size: batch size (75)
    #print_batches: Show model performance per batch (False)
    #n_epochs: number of epochs to train (100)
    #constant_train_flag: (False)
        #Set to true to train based on constant # of epochs per machine 
        #Set to false to train dynamically based on machine performance
        
    #Constant training approach:
        #disc_epochs: Number of consecutive epochs to train discriminator before epoch threshold (5)
        #gen_epochs: Number of consecutive epochs to train generator before epoch threshold (2)
        #epoch_threshold: Epoch number to change training epoch ratio (50)
        #disc_epochs_change: New number of consecutive epochs to train discriminator after epoch threshold is exceeded (1)
        #gen_epochs_change: New number of consecutive epochs to train generator after epoch threshold is exceeded (50)
    
    #Dynamic training approach:                        
        #static_threshold: Epoch number to change from static ratio to dynamic (18)
        #static_disc_epochs: Number of consecutive epochs to train discriminator before epoch threshold (4)
        #static_gen_epochs: Number of consecutive epochs to train generator before epoch threshold (2)
        #pull_threshold: Accuracy threshold for switching machine training when the generator is no longer competitive (0.4)
        #push_threshold: Accuracy threshold for switching machine training when the discriminator is no longer competitive (0.6)
        #recall_threshold: threshold for recall to switch machine training when discriminator is training well
        #switch_flag: indicates if we should switch our training machine (False)
        
def training_loop(X, y, criterion1 = nn.BCELoss(), gan_id = "Mod Test Gan", dig = 5, feature_dim = input_shape, 
                  GENERATOR = 0, DISCRIMINATOR = 1, train_string = "DISC", disc = Discriminator(), z_dim = 100,
                  gen_lr =  0.001, disc_lr = 0.001, batch_size = 100, constant_train_flag = False, disc_epochs = 5,
                  gen_epochs = 2, epoch_threshold = 50, disc_epochs_change = 5, gen_epochs_change = 2, rel_epochs = 0,
                 static_threshold = 28, static_disc_epochs = 5, static_gen_epochs = 2, pull_threshold = 0.3,
                 push_threshold = 0.7, recall_threshold = 0.75, n_epochs = 1000, rows = [],
                switch_flag = False, switch_count = 0, last_real_features = []):
    
    disc.to(device)
    #returns generator, sets discriminator training, creates training tensor, loads data, and initializes optimizers
    gen, to_train, train_features, train_labels, train_data, train_loader, opt_disc, opt_gen = initialize_params(X, y, z_dim, disc_lr, gen_lr, DISCRIMINATOR, batch_size, disc)
    
    #visualize_gen(X, gen, z_dim, a_dim, u_dim)
    gen_epochs = 0
    
    last_D_loss = -1.0
    last_G_loss = -1.0
    
    mean_mean = []
    mean_median = []
    
    for epoch in range(n_epochs):  
        if constant_train_flag:
            if to_train == DISCRIMINATOR and rel_epochs >= disc_epochs:
                rel_epochs = 0
                to_train = GENERATOR
                train_string = "GEN"

            elif to_train == GENERATOR and rel_epochs >= gen_epochs:
                rel_epochs = 0
                to_train = DISCRIMINATOR
                train_string = "DISC"

            # Change epoch ratio after intial 'leveling out'
            if epoch == epoch_threshold:
                rel_epochs = 0
                to_train = GENERATOR
                train_string = "GEN"

                old_ratio = gen_epochs / disc_epochs
                gen_epochs = gen_epochs_change
                disc_epochs = disc_epochs_change
                new_ratio = gen_epochs / disc_epochs
                print(f'\n\nTraining ratio of G/D switched from {old_ratio:.{dig}f} to {new_ratio:.{dig}f}\n\n')
        else:
            if epoch < static_threshold:
                if to_train == DISCRIMINATOR and rel_epochs >= static_disc_epochs:
                    rel_epochs = 0
                    to_train = GENERATOR
                    train_string = "GEN"

                elif to_train == GENERATOR and rel_epochs >= static_gen_epochs:
                    rel_epochs = 0
                    to_train = DISCRIMINATOR
                    train_string = "DISC"

            else:
                if not switch_flag:
                    print("\nSwitching to Dynamic Training\n")
                    switch_flag = True
                    to_train = DISCRIMINATOR
                    train_string = "DISC"
                if to_train == DISCRIMINATOR and fpR <= pull_threshold and R >= recall_threshold:
                    to_train = GENERATOR
                    train_string = "GEN"
                    print("\nPull Generator\n")
                    switch_count += 1
                if to_train == GENERATOR and fpR >= push_threshold:
                    to_train = DISCRIMINATOR
                    train_string = "DISC"
                    print("\nPush Generator\n")
                    switch_count += 1
                    
        print(f'Epoch[{epoch + 1}/{n_epochs}] Train: {train_string} ', end ='')
        
        for batch_idx, (real_features, _) in enumerate(train_loader):
            #Size of this current batch
            batch_size = len(real_features)
            #Send batch to GPU
            real_features = real_features.float().to(device)
            
            if to_train == DISCRIMINATOR:
                ### Training Discriminator
                #visualize_real_batch(real_features.float())
                opt_disc.zero_grad()
                disc_loss = get_disc_loss(gen, disc, criterion1, real_features, batch_size, z_dim)
                disc_loss.backward(retain_graph = True)
                opt_disc.step()
                last_D_loss = disc_loss.item()
                # If generator has yet to ever train, run intial evaluation
                if last_G_loss == -1.0:
                    last_G_loss = get_gen_loss(gen, disc, criterion1, batch_size, z_dim) 
            else:
                ### Training Generator
                opt_gen.zero_grad()
                gen_loss = get_gen_loss(gen, disc, criterion1, batch_size, z_dim)
                gen_loss.backward()
                opt_gen.step()
                last_G_loss = gen_loss.item()
                # If discriminator has yet to ever train, run intial evaluation
                if last_D_loss == -1.0:
                    last_D_loss = get_disc_loss(gen, disc, criterion1, real_features, batch_size, z_dim)
                
            if batch_idx == (len(train_loader) - 1):
                acc, R, fpR = performance_stats(gen, disc, batch_size, z_dim, batch = real_features)
                
        mean_mean_w = torch.mean(torch.Tensor(mean_mean)) 
        mean_median_w = torch.mean(torch.Tensor(mean_median))
            
        ### Currently doesn't print Median/Mean Wasserstein --> Change if needed
        print(f'| LossD: {last_D_loss:.{dig}f}, LossG: {last_G_loss:.{dig}f} | Acc: {acc:.{dig}f} | fpR: {fpR:.{dig}f} | R: {R:.{dig}f}')
    
        if to_train == GENERATOR:
            gen_epochs += 1
            
        mean_mean.clear()
        mean_median.clear()
        rel_epochs += 1
        
        
    print("\n\nTraining Session Finished")
    print(f"Encountered {switch_count} non-trivial training swaps")
    percent = gen_epochs / n_epochs
    print(f"Trained Generator {gen_epochs} out of {n_epochs} ({percent:.3f})")
  
    print("Model Results Sucessfully Saved to \"model_outputs/" + gan_id + ".txt\"")

    save_model(gen, disc, gan_id)
    print("Generator's Trained Parameters Sucessfully Saved to \"saved_models/" + gan_id + "_gen.param\"")
    print("Discriminators's Trained Parameters Sucessfully Saved to \"saved_models/" + gan_id + "_disc.param\"")
    save_model(gen, disc, gan_id)
    
    

In [10]:
X, y = start_data([3], [11], sub_features, act_features) 

#Train
training_loop(X, y, gan_id="U2A1 Solo GAN Group 2", batch_size = len(X), gen_lr=.0001, disc_lr =.0001, n_epochs=20000, dig=5, constant_train_flag=False)

Epoch[1/20000] Train: DISC | LossD: 0.68652, LossG: 0.75076 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[2/20000] Train: DISC | LossD: 0.68631, LossG: 0.75076 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[3/20000] Train: DISC | LossD: 0.68775, LossG: 0.75076 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[4/20000] Train: DISC | LossD: 0.68764, LossG: 0.75076 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[5/20000] Train: DISC | LossD: 0.68871, LossG: 0.75076 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[6/20000] Train: GEN | LossD: 0.68871, LossG: 0.75148 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[7/20000] Train: GEN | LossD: 0.68871, LossG: 0.75112 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[8/20000] Train: DISC | LossD: 0.68471, LossG: 0.75112 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[9/20000] Train: DISC | LossD: 0.68602, LossG: 0.75112 | Acc: 0.50000 | fpR: 0.00000 | R: 0.00000
Epoch[10/20000] Train: DISC | LossD: 0.68331, LossG: 0.75112 | Acc: 0.50000

Epoch[82/20000] Train: DISC | LossD: 0.66755, LossG: 0.74803 | Acc: 0.65217 | fpR: 0.00000 | R: 0.30435
Epoch[83/20000] Train: DISC | LossD: 0.66598, LossG: 0.74803 | Acc: 0.68478 | fpR: 0.00000 | R: 0.36957
Epoch[84/20000] Train: DISC | LossD: 0.66731, LossG: 0.74803 | Acc: 0.71739 | fpR: 0.00000 | R: 0.43478
Epoch[85/20000] Train: DISC | LossD: 0.66298, LossG: 0.74803 | Acc: 0.75000 | fpR: 0.00000 | R: 0.50000
Epoch[86/20000] Train: DISC | LossD: 0.67026, LossG: 0.74803 | Acc: 0.81522 | fpR: 0.00000 | R: 0.63043
Epoch[87/20000] Train: DISC | LossD: 0.67000, LossG: 0.74803 | Acc: 0.82609 | fpR: 0.00000 | R: 0.65217
Epoch[88/20000] Train: DISC | LossD: 0.66747, LossG: 0.74803 | Acc: 0.83696 | fpR: 0.00000 | R: 0.67391
Epoch[89/20000] Train: DISC | LossD: 0.66825, LossG: 0.74803 | Acc: 0.85870 | fpR: 0.00000 | R: 0.71739
Epoch[90/20000] Train: DISC | LossD: 0.66317, LossG: 0.74803 | Acc: 0.89130 | fpR: 0.00000 | R: 0.78261

Pull Generator

Epoch[91/20000] Train: GEN | LossD: 0.66317, Lo

Epoch[160/20000] Train: GEN | LossD: 0.66317, LossG: 0.71648 | Acc: 0.89130 | fpR: 0.00000 | R: 0.78261
Epoch[161/20000] Train: GEN | LossD: 0.66317, LossG: 0.71624 | Acc: 0.89130 | fpR: 0.00000 | R: 0.78261
Epoch[162/20000] Train: GEN | LossD: 0.66317, LossG: 0.71577 | Acc: 0.89130 | fpR: 0.00000 | R: 0.78261
Epoch[163/20000] Train: GEN | LossD: 0.66317, LossG: 0.71431 | Acc: 0.89130 | fpR: 0.00000 | R: 0.78261
Epoch[164/20000] Train: GEN | LossD: 0.66317, LossG: 0.71434 | Acc: 0.88043 | fpR: 0.02174 | R: 0.78261
Epoch[165/20000] Train: GEN | LossD: 0.66317, LossG: 0.71603 | Acc: 0.88043 | fpR: 0.02174 | R: 0.78261
Epoch[166/20000] Train: GEN | LossD: 0.66317, LossG: 0.71348 | Acc: 0.89130 | fpR: 0.00000 | R: 0.78261
Epoch[167/20000] Train: GEN | LossD: 0.66317, LossG: 0.71361 | Acc: 0.88043 | fpR: 0.02174 | R: 0.78261
Epoch[168/20000] Train: GEN | LossD: 0.66317, LossG: 0.71231 | Acc: 0.89130 | fpR: 0.00000 | R: 0.78261
Epoch[169/20000] Train: GEN | LossD: 0.66317, LossG: 0.71386 | A

Epoch[239/20000] Train: GEN | LossD: 0.66317, LossG: 0.68782 | Acc: 0.71739 | fpR: 0.34783 | R: 0.78261
Epoch[240/20000] Train: GEN | LossD: 0.66317, LossG: 0.68771 | Acc: 0.63043 | fpR: 0.52174 | R: 0.78261
Epoch[241/20000] Train: GEN | LossD: 0.66317, LossG: 0.68524 | Acc: 0.60870 | fpR: 0.56522 | R: 0.78261
Epoch[242/20000] Train: GEN | LossD: 0.66317, LossG: 0.68762 | Acc: 0.67391 | fpR: 0.43478 | R: 0.78261
Epoch[243/20000] Train: GEN | LossD: 0.66317, LossG: 0.68606 | Acc: 0.63043 | fpR: 0.52174 | R: 0.78261
Epoch[244/20000] Train: GEN | LossD: 0.66317, LossG: 0.68627 | Acc: 0.66304 | fpR: 0.45652 | R: 0.78261
Epoch[245/20000] Train: GEN | LossD: 0.66317, LossG: 0.68587 | Acc: 0.70652 | fpR: 0.36957 | R: 0.78261
Epoch[246/20000] Train: GEN | LossD: 0.66317, LossG: 0.68551 | Acc: 0.65217 | fpR: 0.47826 | R: 0.78261
Epoch[247/20000] Train: GEN | LossD: 0.66317, LossG: 0.68457 | Acc: 0.63043 | fpR: 0.52174 | R: 0.78261
Epoch[248/20000] Train: GEN | LossD: 0.66317, LossG: 0.68419 | A

Epoch[324/20000] Train: GEN | LossD: 0.67620, LossG: 0.68171 | Acc: 0.67391 | fpR: 0.65217 | R: 1.00000
Epoch[325/20000] Train: GEN | LossD: 0.67620, LossG: 0.68116 | Acc: 0.70652 | fpR: 0.58696 | R: 1.00000
Epoch[326/20000] Train: GEN | LossD: 0.67620, LossG: 0.68137 | Acc: 0.66304 | fpR: 0.67391 | R: 1.00000
Epoch[327/20000] Train: GEN | LossD: 0.67620, LossG: 0.68104 | Acc: 0.63043 | fpR: 0.73913 | R: 1.00000

Push Generator

Epoch[328/20000] Train: DISC | LossD: 0.68130, LossG: 0.68104 | Acc: 0.67391 | fpR: 0.65217 | R: 1.00000
Epoch[329/20000] Train: DISC | LossD: 0.68360, LossG: 0.68104 | Acc: 0.65217 | fpR: 0.69565 | R: 1.00000
Epoch[330/20000] Train: DISC | LossD: 0.67790, LossG: 0.68104 | Acc: 0.75000 | fpR: 0.50000 | R: 1.00000
Epoch[331/20000] Train: DISC | LossD: 0.67821, LossG: 0.68104 | Acc: 0.70652 | fpR: 0.58696 | R: 1.00000
Epoch[332/20000] Train: DISC | LossD: 0.68305, LossG: 0.68104 | Acc: 0.67391 | fpR: 0.65217 | R: 1.00000
Epoch[333/20000] Train: DISC | LossD: 0.68

...

Epoch[19604/20000] Train: GEN | LossD: 0.64343, LossG: 0.72586 | Acc: 0.56522 | fpR: 0.69565 | R: 0.82609
Epoch[19605/20000] Train: GEN | LossD: 0.64343, LossG: 0.70233 | Acc: 0.56522 | fpR: 0.69565 | R: 0.82609
Epoch[19606/20000] Train: GEN | LossD: 0.64343, LossG: 0.68106 | Acc: 0.54348 | fpR: 0.73913 | R: 0.82609

Push Generator

Epoch[19607/20000] Train: DISC | LossD: 0.67366, LossG: 0.68106 | Acc: 0.55435 | fpR: 0.71739 | R: 0.82609
Epoch[19608/20000] Train: DISC | LossD: 0.69171, LossG: 0.68106 | Acc: 0.64130 | fpR: 0.56522 | R: 0.84783
Epoch[19609/20000] Train: DISC | LossD: 0.66499, LossG: 0.68106 | Acc: 0.60870 | fpR: 0.63043 | R: 0.84783
Epoch[19610/20000] Train: DISC | LossD: 0.69021, LossG: 0.68106 | Acc: 0.65217 | fpR: 0.54348 | R: 0.84783
Epoch[19611/20000] Train: DISC | LossD: 0.65736, LossG: 0.68106 | Acc: 0.67391 | fpR: 0.47826 | R: 0.82609
Epoch[19612/20000] Train: DISC | LossD: 0.65336, LossG: 0.68106 | Acc: 0.68478 | fpR: 0.43478 | R: 0.80435
Epoch[19613/20000] Trai

Epoch[19696/20000] Train: GEN | LossD: 0.65848, LossG: 0.73652 | Acc: 0.75000 | fpR: 0.28261 | R: 0.78261
Epoch[19697/20000] Train: GEN | LossD: 0.65848, LossG: 0.72387 | Acc: 0.65217 | fpR: 0.47826 | R: 0.78261
Epoch[19698/20000] Train: GEN | LossD: 0.65848, LossG: 0.72030 | Acc: 0.67391 | fpR: 0.43478 | R: 0.78261
Epoch[19699/20000] Train: GEN | LossD: 0.65848, LossG: 0.70816 | Acc: 0.57609 | fpR: 0.63043 | R: 0.78261
Epoch[19700/20000] Train: GEN | LossD: 0.65848, LossG: 0.70639 | Acc: 0.54348 | fpR: 0.69565 | R: 0.78261
Epoch[19701/20000] Train: GEN | LossD: 0.65848, LossG: 0.67680 | Acc: 0.60870 | fpR: 0.56522 | R: 0.78261
Epoch[19702/20000] Train: GEN | LossD: 0.65848, LossG: 0.69675 | Acc: 0.54348 | fpR: 0.69565 | R: 0.78261
Epoch[19703/20000] Train: GEN | LossD: 0.65848, LossG: 0.68409 | Acc: 0.52174 | fpR: 0.73913 | R: 0.78261

Push Generator

Epoch[19704/20000] Train: DISC | LossD: 0.69716, LossG: 0.68409 | Acc: 0.53261 | fpR: 0.71739 | R: 0.78261
Epoch[19705/20000] Train: DI

Epoch[19784/20000] Train: DISC | LossD: 0.69789, LossG: 0.66519 | Acc: 0.57609 | fpR: 0.71739 | R: 0.86957
Epoch[19785/20000] Train: DISC | LossD: 0.70607, LossG: 0.66519 | Acc: 0.55435 | fpR: 0.76087 | R: 0.86957
Epoch[19786/20000] Train: DISC | LossD: 0.69175, LossG: 0.66519 | Acc: 0.57609 | fpR: 0.69565 | R: 0.84783
Epoch[19787/20000] Train: DISC | LossD: 0.68259, LossG: 0.66519 | Acc: 0.64130 | fpR: 0.54348 | R: 0.82609
Epoch[19788/20000] Train: DISC | LossD: 0.69765, LossG: 0.66519 | Acc: 0.61957 | fpR: 0.56522 | R: 0.80435
Epoch[19789/20000] Train: DISC | LossD: 0.69022, LossG: 0.66519 | Acc: 0.63043 | fpR: 0.54348 | R: 0.80435
Epoch[19790/20000] Train: DISC | LossD: 0.68233, LossG: 0.66519 | Acc: 0.65217 | fpR: 0.50000 | R: 0.80435
Epoch[19791/20000] Train: DISC | LossD: 0.66729, LossG: 0.66519 | Acc: 0.75000 | fpR: 0.28261 | R: 0.78261

Pull Generator

Epoch[19792/20000] Train: GEN | LossD: 0.66729, LossG: 0.70815 | Acc: 0.68478 | fpR: 0.41304 | R: 0.78261
Epoch[19793/20000] Tr

Epoch[19880/20000] Train: DISC | LossD: 0.69368, LossG: 0.67157 | Acc: 0.59783 | fpR: 0.63043 | R: 0.82609
Epoch[19881/20000] Train: DISC | LossD: 0.68085, LossG: 0.67157 | Acc: 0.60870 | fpR: 0.60870 | R: 0.82609
Epoch[19882/20000] Train: DISC | LossD: 0.67464, LossG: 0.67157 | Acc: 0.58696 | fpR: 0.65217 | R: 0.82609
Epoch[19883/20000] Train: DISC | LossD: 0.67776, LossG: 0.67157 | Acc: 0.70652 | fpR: 0.41304 | R: 0.82609
Epoch[19884/20000] Train: DISC | LossD: 0.67823, LossG: 0.67157 | Acc: 0.59783 | fpR: 0.63043 | R: 0.82609
Epoch[19885/20000] Train: DISC | LossD: 0.66656, LossG: 0.67157 | Acc: 0.61957 | fpR: 0.56522 | R: 0.80435
Epoch[19886/20000] Train: DISC | LossD: 0.66369, LossG: 0.67157 | Acc: 0.71739 | fpR: 0.36957 | R: 0.80435
Epoch[19887/20000] Train: DISC | LossD: 0.64542, LossG: 0.67157 | Acc: 0.67391 | fpR: 0.43478 | R: 0.78261
Epoch[19888/20000] Train: DISC | LossD: 0.65940, LossG: 0.67157 | Acc: 0.75000 | fpR: 0.26087 | R: 0.76087

Pull Generator

Epoch[19889/20000] T

Epoch[19970/20000] Train: DISC | LossD: 0.68213, LossG: 0.68666 | Acc: 0.66304 | fpR: 0.43478 | R: 0.76087
Epoch[19971/20000] Train: DISC | LossD: 0.69123, LossG: 0.68666 | Acc: 0.65217 | fpR: 0.47826 | R: 0.78261
Epoch[19972/20000] Train: DISC | LossD: 0.69171, LossG: 0.68666 | Acc: 0.63043 | fpR: 0.52174 | R: 0.78261
Epoch[19973/20000] Train: DISC | LossD: 0.67762, LossG: 0.68666 | Acc: 0.65217 | fpR: 0.50000 | R: 0.80435
Epoch[19974/20000] Train: DISC | LossD: 0.68364, LossG: 0.68666 | Acc: 0.67391 | fpR: 0.45652 | R: 0.80435
Epoch[19975/20000] Train: DISC | LossD: 0.68693, LossG: 0.68666 | Acc: 0.63043 | fpR: 0.54348 | R: 0.80435
Epoch[19976/20000] Train: DISC | LossD: 0.68460, LossG: 0.68666 | Acc: 0.60870 | fpR: 0.58696 | R: 0.80435
Epoch[19977/20000] Train: DISC | LossD: 0.67617, LossG: 0.68666 | Acc: 0.66304 | fpR: 0.47826 | R: 0.80435
Epoch[19978/20000] Train: DISC | LossD: 0.66440, LossG: 0.68666 | Acc: 0.64130 | fpR: 0.52174 | R: 0.80435
Epoch[19979/20000] Train: DISC | Loss